#   형태소 분석

In [3]:
from konlpy.tag import Twitter

twitter = Twitter()

malist = twitter.pos("아버지 가방에 들어가신다.", norm=True, stem=True)
print(malist)

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가다', 'Verb'), ('.', 'Punctuation')]


#  출현 빈도 분석

In [ ]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter

fp = codecs.open()

# 위키피디아 한국어 버전을 사전으로 사용해보기

In [7]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from gensim.models import word2vec

readFp = codecs.open("python-for-machine-learning_20190826/위키피디아모델/wiki.txt", "r", encoding="utf-8")
wakati_file = "wiki.wakati"
writeFp = open(wakati_file, "w", encoding="utf-8")

twitter = Twitter()
i = 0

while True:
    line = readFp.readline()
    if not line: break
    if i % 20000 == 0:
        print("current - " + str(i))
    
    i += 1
    
    malist = twitter.pos(line, norm=True, stem=True)
    r = []
    for word in malist:
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            writeFp.write(word[0] + " ")
writeFp.close()

current - 0
current - 20000


KeyboardInterrupt: 

In [10]:
from gensim.models import word2vec
data = word2vec.Text8Corpus("python-for-machine-learning_20190826/위키피디아모델/wiki.wakati")
model = word2vec.Word2Vec(data, vector_size=100)
model.save("wiki.model")
print("ok")

ok


# 레벤슈타인 거리를 계산하는 프로그램

In [2]:
def calc_distance(a, b):
    ''' 레벤슈타인 거리 계산하기'''
    if a == b: return 0
    a_len = len(a)
    b_len = len(b)
    if a == "": return b_len
    if b == "": return a_len
    # 2차원 표 (a_len+1, b_len+1) 준비하기
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1):
        matrix[i] = [0 for j in range(b_len+1)]
    
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                                matrix[i-1][j] + 1,
                                matrix[i][j-1] + 1,
                                matrix[i-1][j-1] + cost
                               ])
    return matrix[a_len][b_len]

print(calc_distance("가나다라", "가마바라"))

samples = ["신촌역", "신천군", "신천역", "신발", "마곡역"]
base = samples[0]
r = sorted(samples, key = lambda n:calc_distance(base, n))
for n in r:
    print(calc_distance(base, n), n)

2
0 신촌역
1 신천역
2 신천군
2 신발
2 마곡역


# N-gram으로 유사도 구하기

In [4]:
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt / len(a), r

a = "오늘 강남에서 맛있는 스파게티를 먹었다."
b = "강남에서 먹었던 오늘의 스파게티는 맛있었다."

r2, word2 = diff_ngram(a, b, 2)
print("2-gram:", r2, word2)

r3, word3 = diff_ngram(a, b, 3)
print("3-gram:", r3, word3)

2-gram: 0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']
3-gram: 0.45 ['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']
